Proyecto mineria de datos:
### Minería de Datos para Equidad Educativa en Comunas Vulnerables
##### grupo: 14

##### Integrantes: 
1) Ignacio Gabler  
2) Cristobla Merino


### 1) Preprocesamiento de Datos

In [59]:
import zipfile
import os
import pathlib

# Obtener la ruta del directorio actual
ruta_actual = pathlib.Path().absolute()

# Ruta relativa al archivo zip
ruta_zip = os.path.join(ruta_actual, "archive.zip")
# Carpeta destino
carpeta_destino = os.path.join(ruta_actual, "datos")

# Descomprimir
with zipfile.ZipFile(ruta_zip, 'r') as zip_ref:
    zip_ref.extractall(carpeta_destino)



print(f"archivos extraidos en: {carpeta_destino}")

archivos extraidos en: c:\Users\ignac\OneDrive\Desktop\md14_proyecto\datos


In [60]:
#vamos a utilizarel archivo Expanded_data_with_more_features.csv porque es 
# el que contiene más características que nos interesa

import pandas as pd	
# Cargar el dataset desde archivo local (ajusta el nombre si es distinto)
df = pd.read_csv("datos/Expanded_data_with_more_features.csv")

# Ver las primeras filas del dataset
print(df.head())

# Ver resumen de columnas y tipos de datos
print(df.info())

# Ver si hay valores faltantes (NaN)
print(df.isnull().sum())


   Unnamed: 0  Gender EthnicGroup          ParentEduc     LunchType TestPrep  \
0           0  female         NaN   bachelor's degree      standard     none   
1           1  female     group C        some college      standard      NaN   
2           2  female     group B     master's degree      standard     none   
3           3    male     group A  associate's degree  free/reduced     none   
4           4    male     group C        some college      standard     none   

  ParentMaritalStatus PracticeSport IsFirstChild  NrSiblings TransportMeans  \
0             married     regularly          yes         3.0     school_bus   
1             married     sometimes          yes         0.0            NaN   
2              single     sometimes          yes         4.0     school_bus   
3             married         never           no         1.0            NaN   
4             married     sometimes          yes         0.0     school_bus   

  WklyStudyHours  MathScore  ReadingScore  W

In [61]:
df_copy = df.copy()

# eliminar null
df_copy = df_copy.dropna()
# Verificar si se eliminaron filas con valores nulos
print(f"Filas antes de eliminar nulos: {len(df)}")
print(f"Filas después de eliminar nulos: {len(df_copy)}")

Filas antes de eliminar nulos: 30641
Filas después de eliminar nulos: 19243


Eliminamos las filas con valores nulos, pero no borramos niguna columna (excepto la primera de id) para no perder información relevante. De esta manera obtener los factores y habitos que afectan en el rendimiento estudiantil de los alumnos de las comunas vulnerables de la región metropolitana.





In [62]:
#dropear unammed
df_copy = df_copy.drop(columns=['Unnamed: 0'], errors='ignore')

### 2) Encoding variables categoricas

#### label encoding

In [63]:

df_copy['Gender'] = df_copy['Gender'].map({'female': 0, 'male': 1})

# yes = 1  , no = 0
df_copy['IsFirstChild'] = df_copy['IsFirstChild'].map({'yes': 1, 'no': 0})



In [64]:

df_copy['TestPrep'] = df_copy['TestPrep'].map({'none': 0, 'completed': 1})
df_copy['PracticeSport'] = df_copy['PracticeSport'].map({'never': 0, 'sometimes': 1, 'regularly': 2})


#### One hot encoding

In [65]:
df_copy = pd.get_dummies(df_copy, columns=['EthnicGroup', 'ParentEduc', 'LunchType', 'ParentMaritalStatus', 'TransportMeans', 'WklyStudyHours'], drop_first=True)


In [66]:
df_copy = df_copy.astype(int)

print(df_copy.head(1))

# quiero ver todas los nombres de columnas
print("Nombres de columnas:")
print(df_copy.columns.tolist())

   Gender  TestPrep  PracticeSport  IsFirstChild  NrSiblings  MathScore  \
2       0         0              1             1           4         87   

   ReadingScore  WritingScore  EthnicGroup_group B  EthnicGroup_group C  ...  \
2            93            91                    1                    0  ...   

   ParentEduc_master's degree  ParentEduc_some college  \
2                           1                        0   

   ParentEduc_some high school  LunchType_standard  \
2                            0                   1   

   ParentMaritalStatus_married  ParentMaritalStatus_single  \
2                            0                           1   

   ParentMaritalStatus_widowed  TransportMeans_school_bus  WklyStudyHours_< 5  \
2                            0                          1                   1   

   WklyStudyHours_> 10  
2                    0  

[1 rows x 24 columns]
Nombres de columnas:
['Gender', 'TestPrep', 'PracticeSport', 'IsFirstChild', 'NrSiblings', 'MathScore

#### Variables Objetivo

In [69]:
from sklearn.preprocessing import StandardScaler

# Separar variables de entrada y objetivo
y = df_copy[['MathScore', 'ReadingScore', 'WritingScore']]
X = df_copy.drop(columns=['MathScore', 'ReadingScore', 'WritingScore'])
 

# Escalar variables
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
